In [ ]:
from toolkit.lap.track import *
from toolkit.lap.channels import *
from toolkit.lap.gps import *
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import rerun as rr


In [ ]:
raw_track = load_track_from_mat('./data/lap_data/2017_Michigan_Endurance_First4Laps.mat')
data_dict = parse_car_data_mat(raw_track)
track_gps = gps_from_channels(data_dict["GPS_Latitude"], data_dict["GPS_Longitude"], data_dict["GPS_Altitude"], data_dict["GPS_Time"])

In [ ]:
data_dict.keys()

In [ ]:
data_dict["GPS_Speed"].unit

In [ ]:
data_dict["__gps_vel"] = unit_conversion(data_dict["GPS_Speed"], "m/s", 1/MS_TO_MPH)
data_dict["__ws_fl"] = unit_conversion(data_dict["Wheel_Speed_FL"], "m/s", 1/MS_TO_MPH, "Wheel Speed FL", "WS_FL")
data_dict["__ws_fr"] = unit_conversion(data_dict["Wheel_Speed_FR"], "m/s", 1/MS_TO_MPH, "Wheel Speed FR", "WS_FR")
data_dict["__ws_rl"] = unit_conversion(data_dict["Wheel_Speed_RL"], "m/s", 1/MS_TO_MPH, "Wheel Speed RL", "WS_RL")
data_dict["__ws_rr"] = unit_conversion(data_dict["Wheel_Speed_RR"], "m/s", 1/MS_TO_MPH, "Wheel Speed RR", "WS_RR")
mr = 1.5
data_dict["__nl_fl"] = unit_conversion(data_dict["Load_Cell__Force_FL"], "N", LB_TO_KG * G / mr, "Normal Load Force FL", "NL_FL")
data_dict["__nl_fr"] = unit_conversion(data_dict["Load_Cell_Force_FR"], "N", LB_TO_KG * G / mr, "Normal Load Force FR", "NL_FR")
data_dict["__nl_rl"] = unit_conversion(data_dict["Load_Cell_Force_RL"], "N", LB_TO_KG * G * -1 / mr, "Normal Load Force RL", "NL_RL")
data_dict["__nl_rr"] = unit_conversion(data_dict["Load_Cell_Force_RR"], "N", LB_TO_KG * G * -1 / mr, "Normal Load Force RR", "NL_RR")
data_dict["__steering_angle"] = deg2rad_chan(data_dict["Steering_Wheel_Angle"])
data_dict["__acc_x"] = data_dict["G_Force_Lat"]
data_dict["__acc_y"] = data_dict["G_Force_Vert"]
data_dict["__acc_y"].data += -1
data_dict["__acc_z"] = data_dict["G_Force_Long"]
data_dict["__gyro_x"] = deg2rad_chan(data_dict["Vehicle_Yaw_Rate"])
data_dict["__yacc"] = derivative_chan(data_dict["__gyro_x"], "Yaw Acceleration", "__yacc", "rad/s^2", 20)


In [ ]:

track = Track(track_gps, data_dict, 50, 0.95)

In [ ]:
def display_track(a_track):
    rr.init("log viewer", spawn=True)
    for name in WELL_KNOWN_KEYS:
        time, data, p_name = a_track.get_channel(name)
        rr.send_columns(
            name,
            times=[rr.TimeSecondsColumn("step", time)],
            components=[rr.components.ScalarBatch(data)]
        )
    rr.log("track", rr.LineStrips2D(a_track.smooth_gps.get_2d_track(enu=True), radii=0.002, colors=[0, 255, 255, 255]), timeless=True)
    rr.send_columns(
        "gps",
        times=[rr.TimeSecondsColumn("step", a_track.smooth_gps.time)],
        components=[rr.Points2D.indicator(),
        rr.components.Position2DBatch(a_track.smooth_gps.get_2d_track(enu=True)),]
    )


In [ ]:
display_track(track)